In [1]:
#Importing libraries 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import re
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Reading test data 
test=pd.read_csv("test.csv")

In [3]:
test.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance
0,DEN_XNA_62349,2005,9,12,1,1516,1756,100,DEN,XNA,616
1,RIC_EWR_87213,2005,11,14,1,1300,1415,75,RIC,EWR,278
2,IAD_GSO_86576,2005,9,28,3,1040,1144,64,IAD,GSO,239
3,SAV_CVG_87264,2005,11,18,5,1120,1302,102,SAV,CVG,515
4,SBP_LAX_68158,2005,7,5,2,630,733,63,SBP,LAX,155


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6566 entries, 0 to 6565
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   FlightNumber         6566 non-null   object
 1   Year                 6566 non-null   int64 
 2   Month                6566 non-null   int64 
 3   DayofMonth           6566 non-null   int64 
 4   DayOfWeek            6566 non-null   int64 
 5   ScheduledDepTime     6566 non-null   int64 
 6   ScheduledArrTime     6566 non-null   int64 
 7   ScheduledTravelTime  6566 non-null   int64 
 8   Origin               6566 non-null   object
 9   Destination          6566 non-null   object
 10  Distance             6566 non-null   int64 
dtypes: int64(8), object(3)
memory usage: 564.4+ KB


In [5]:
#Converting time to datetime in train data set
test.ScheduledDepTime = test.ScheduledDepTime.astype(str).str.zfill(4)
test.ScheduledArrTime = test.ScheduledArrTime.astype(str).str.zfill(4)
test.ScheduledDepTime = pd.to_datetime(test.ScheduledDepTime, format="%H%M").dt.time
test.ScheduledArrTime = pd.to_datetime(test.ScheduledArrTime, format="%H%M").dt.time

In [6]:
test.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance
0,DEN_XNA_62349,2005,9,12,1,15:16:00,17:56:00,100,DEN,XNA,616
1,RIC_EWR_87213,2005,11,14,1,13:00:00,14:15:00,75,RIC,EWR,278
2,IAD_GSO_86576,2005,9,28,3,10:40:00,11:44:00,64,IAD,GSO,239
3,SAV_CVG_87264,2005,11,18,5,11:20:00,13:02:00,102,SAV,CVG,515
4,SBP_LAX_68158,2005,7,5,2,06:30:00,07:33:00,63,SBP,LAX,155


In [7]:
test.DayofMonth = test.DayofMonth.astype(str).str.zfill(2)
test.Month = test.Month.astype(str).str.zfill(2)
test.Year = test.Year.astype(str).str.zfill(4)

In [8]:
test.Year = pd.to_datetime(test.Year + "/" + test.Month + "/" + test.DayofMonth)

In [9]:
test.Year = test.apply(lambda r : pd.datetime.combine(r['Year'],r['ScheduledDepTime']),1)

In [10]:
test.rename(columns = {'Year':'Date'}, inplace = True)

In [11]:
test.head()

,FlightNumber,Date,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance
0,DEN_XNA_62349,2005-09-12 15:16:00,09,12,1,15:16:00,17:56:00,100,DEN,XNA,616
1,RIC_EWR_87213,2005-11-14 13:00:00,11,14,1,13:00:00,14:15:00,75,RIC,EWR,278
2,IAD_GSO_86576,2005-09-28 10:40:00,09,28,3,10:40:00,11:44:00,64,IAD,GSO,239
3,SAV_CVG_87264,2005-11-18 11:20:00,11,18,5,11:20:00,13:02:00,102,SAV,CVG,515
4,SBP_LAX_68158,2005-07-05 06:30:00,07,05,2,06:30:00,07:33:00,63,SBP,LAX,155


In [12]:
#Drop Month, Dayofmonth
#Drop scheduleddeptime from test data because we have scheduleddeptime is in date
test=test.drop(['Month', 'DayofMonth','ScheduledDepTime'],axis = 1)
test.head()

,FlightNumber,Date,DayOfWeek,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance
0,DEN_XNA_62349,2005-09-12 15:16:00,1,17:56:00,100,DEN,XNA,616
1,RIC_EWR_87213,2005-11-14 13:00:00,1,14:15:00,75,RIC,EWR,278
2,IAD_GSO_86576,2005-09-28 10:40:00,3,11:44:00,64,IAD,GSO,239
3,SAV_CVG_87264,2005-11-18 11:20:00,5,13:02:00,102,SAV,CVG,515
4,SBP_LAX_68158,2005-07-05 06:30:00,2,07:33:00,63,SBP,LAX,155


In [13]:
#Rename Date as Departure Time
test.rename(columns = {'Date':'DepartureTime'}, inplace = True)
test.head()

,FlightNumber,DepartureTime,DayOfWeek,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance
0,DEN_XNA_62349,2005-09-12 15:16:00,1,17:56:00,100,DEN,XNA,616
1,RIC_EWR_87213,2005-11-14 13:00:00,1,14:15:00,75,RIC,EWR,278
2,IAD_GSO_86576,2005-09-28 10:40:00,3,11:44:00,64,IAD,GSO,239
3,SAV_CVG_87264,2005-11-18 11:20:00,5,13:02:00,102,SAV,CVG,515
4,SBP_LAX_68158,2005-07-05 06:30:00,2,07:33:00,63,SBP,LAX,155


In [14]:
# Reading All stataions data 
Allstationsdata=pd.read_csv("AllStationsData_PHD.txt",sep='|')

In [15]:
Allstationsdata.head()

,WeatherStationID,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,3011,TEX,0,0,0,37.57,-107.55,+7
1,3012,SKX,0,0,0,36.28,-105.40,+7
2,3013,LAA,3683,3703,3675,38.04,-102.41,+7
3,3014,4SL,0,6909,0,35.47,-107.14,+7
4,3016,RIL,5503,5544,5506,39.32,-107.44,+7


In [16]:
Allstationsdata.TimeZone.unique()

array(['+7', '+6', '+8', '+5', '+4', '+9', '+10', 'Yukon Standard', '-10',
       'Bering Standard', 'Alaska-Hawaii', 'Eastern Standard'],
      dtype=object)

In [17]:
Allstationsdata.rmtimezone = ['-10', 'Yukon Standard','Bering Standard','Alaska-Hawaii','Eastern Standard']
Allstationsdata= Allstationsdata[~Allstationsdata.TimeZone.isin(Allstationsdata.rmtimezone)]
Allstationsdata= Allstationsdata.reset_index(drop=True)

In [18]:
Allstationsdata.TimeZone.unique()

array(['+7', '+6', '+8', '+5', '+4', '+9', '+10'], dtype=object)

In [19]:
Allstationsdata.TimeZone= pd.to_numeric(Allstationsdata.TimeZone)

In [20]:
Allstationsdata.TimeZone =  pd.TimedeltaIndex(Allstationsdata.TimeZone, unit='h')

In [21]:
test.info()
Allstationsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6566 entries, 0 to 6565
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   FlightNumber         6566 non-null   object        
 1   DepartureTime        6566 non-null   datetime64[ns]
 2   DayOfWeek            6566 non-null   int64         
 3   ScheduledArrTime     6566 non-null   object        
 4   ScheduledTravelTime  6566 non-null   int64         
 5   Origin               6566 non-null   object        
 6   Destination          6566 non-null   object        
 7   Distance             6566 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 410.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   WeatherStationID  1730 non-null 

In [22]:
# Converting ScheduledDepTime to UTC dep time
a=0
for i in test.Origin:
    b=0
    for j in Allstationsdata.AirportID:
        if (i==j): 
            #print ("Matchfound       Origin:",i," DepartureTime:",test.DepartureTime[a],"    index(a):",a)
            #print("              AirportID:",j,"   tzone:",Allstationsdata.TimeZone[b],"        index(b):",b)
            #for k in range(80): print("_",end='')
            #print("")
            test.DepartureTime[a] = test.DepartureTime[a] + Allstationsdata.TimeZone[b]
        b=b+1               
    a=a+1

In [23]:
test.rename(columns = {'DepartureTime':'UTC_Deptime'}, inplace = True)
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance
0,DEN_XNA_62349,2005-09-12 22:16:00,1,17:56:00,100,DEN,XNA,616
1,RIC_EWR_87213,2005-11-14 18:00:00,1,14:15:00,75,RIC,EWR,278
2,IAD_GSO_86576,2005-09-28 15:40:00,3,11:44:00,64,IAD,GSO,239
3,SAV_CVG_87264,2005-11-18 16:20:00,5,13:02:00,102,SAV,CVG,515
4,SBP_LAX_68158,2005-07-05 14:30:00,2,07:33:00,63,SBP,LAX,155


In [24]:
# Converting ScheduledArrTime to UTC arrival time
a=0
for i in test.UTC_Deptime:
    test.ScheduledArrTime[a] = test.UTC_Deptime[a] + pd.Timedelta(minutes=test.ScheduledTravelTime[a])
    a=a+1

In [25]:
test.rename(columns = {'ScheduledArrTime':'UTC_Arrtime'}, inplace = True)
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616
1,RIC_EWR_87213,2005-11-14 18:00:00,1,2005-11-14 19:15:00,75,RIC,EWR,278
2,IAD_GSO_86576,2005-09-28 15:40:00,3,2005-09-28 16:44:00,64,IAD,GSO,239
3,SAV_CVG_87264,2005-11-18 16:20:00,5,2005-11-18 18:02:00,102,SAV,CVG,515
4,SBP_LAX_68158,2005-07-05 14:30:00,2,2005-07-05 15:33:00,63,SBP,LAX,155


In [26]:
# Merging all station data with test data
# Using origin 
test["Origin_ws_id"] = 0
test["Destination_ws_id"] = 0

In [27]:
a=0
for i in test.Origin:
    b=0
    for j in Allstationsdata.AirportID:
        if (i==j): 
            test["Origin_ws_id"][a] = Allstationsdata.WeatherStationID[b]
        b=b+1               
    a=a+1

In [28]:
a=0
for i in test.Origin:
    b=0
    for j in Allstationsdata.AirportID:
        if (i==j): 
            test["Destination_ws_id"][a] = Allstationsdata.WeatherStationID[b]
        b=b+1               
    a=a+1

In [29]:
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017
1,RIC_EWR_87213,2005-11-14 18:00:00,1,2005-11-14 19:15:00,75,RIC,EWR,278,13740,13740
2,IAD_GSO_86576,2005-09-28 15:40:00,3,2005-09-28 16:44:00,64,IAD,GSO,239,93738,93738
3,SAV_CVG_87264,2005-11-18 16:20:00,5,2005-11-18 18:02:00,102,SAV,CVG,515,3822,3822
4,SBP_LAX_68158,2005-07-05 14:30:00,2,2005-07-05 15:33:00,63,SBP,LAX,155,93206,93206


In [30]:
# Merging all station data with test data
# Using origin 
test = test.merge(Allstationsdata, left_on = "Origin_ws_id" ,right_on = "WeatherStationID")

In [31]:
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,WeatherStationID,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00


In [32]:
# Drop longitude,timezone,latitude,airport id, weatherstation id from test data
test = test.drop(['Longitude','TimeZone','Latitude','AirportID','WeatherStationID'],axis = 1)


In [33]:
# Rename columns
test.rename(columns = {"GroundHeight":"Origin_GroH","StationHeight":"Origin_StaH",
                        "BarometerHeight":"Origin_BaroH"}, inplace = True)
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382


In [34]:
# Merging all station data with test data
# Using origin 
test = test.merge(Allstationsdata, left_on = "Destination_ws_id" ,right_on = "WeatherStationID")

In [35]:
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,WeatherStationID,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,3017,DEN,5379,5431,5382,39.5,-104.4,0 days 07:00:00


In [36]:
# Drop longitude,timezone,latitude,airport id, weatherstation id from test data
test = test.drop(['Longitude','TimeZone','Latitude','AirportID','WeatherStationID'],axis = 1)

In [37]:
# Rename columns
test.rename(columns = {"GroundHeight":"Des_GroH","StationHeight":"Des_StaH",
                        "BarometerHeight":"Des_BaroH"}, inplace = True)
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382


In [38]:
#Reading 2005 precipitation hpd data
p7=pd.read_csv("200503hpd.txt")
p8=pd.read_csv("200507hpd.txt")
p9=pd.read_csv("200509hpd.txt")
p10=pd.read_csv("200511hpd.txt")

In [39]:
p_2005=[p7,p8,p9,p10]
precipitation_2005=pd.concat(p_2005)
precipitation_2005=precipitation_2005.reset_index(drop=True)

In [40]:
precipitation_2005.describe().apply(lambda s: s.apply('{0:0.2f}'.format))

,WeatherStationID,YearMonthDay,Time,HourlyPrecip
count,4410021.00,4410021.00,4410021.00,4266127.00
mean,37009.32,20050771.52,1201.22,0.00
std,33486.11,293.03,686.51,0.05
min,3011.00,20050301.00,10.00,0.00
25%,12904.00,20050702.00,645.00,0.00
50%,23254.00,20050901.00,1156.00,0.00
75%,54819.00,20051101.00,1756.00,0.00
max,96402.00,20051130.00,2359.00,60.08


In [41]:
precipitation_2005.Time = precipitation_2005.Time.astype(str).str.zfill(4)
precipitation_2005.Time = pd.to_datetime(precipitation_2005.Time, format="%H%M").dt.time
precipitation_2005.head()

,WeatherStationID,YearMonthDay,Time,HourlyPrecip
0,27401,20050301,00:15:00,0.0
1,24201,20050301,00:17:00,0.0
2,13944,20050301,00:30:00,0.0
3,14704,20050301,00:34:00,0.0
4,27401,20050301,00:35:00,0.0


In [42]:
precipitation_2005.YearMonthDay = pd.to_datetime(precipitation_2005.YearMonthDay, format="%Y%m%d")
precipitation_2005.Time = precipitation_2005.apply(lambda r : pd.datetime.combine(r['YearMonthDay'],r['Time']),1)
precipitation_2005 =precipitation_2005.drop(['YearMonthDay'],axis = 1)

In [43]:
precipitation_2005.head()

,WeatherStationID,Time,HourlyPrecip
0,27401,2005-03-01 00:15:00,0.0
1,24201,2005-03-01 00:17:00,0.0
2,13944,2005-03-01 00:30:00,0.0
3,14704,2005-03-01 00:34:00,0.0
4,27401,2005-03-01 00:35:00,0.0


In [44]:
# Merging precipitation data with all station data
precipitation_2005=precipitation_2005.merge(Allstationsdata, how='left', on=['WeatherStationID'])
precipitation_2005.head()

,WeatherStationID,Time,HourlyPrecip,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,27401,2005-03-01 00:15:00,0.0,BTI,0.0,5.0,0.0,70.08,-143.35,0 days 09:00:00
1,24201,2005-03-01 00:17:00,0.0,GRF,-8.0,302.0,0.0,47.05,-122.35,0 days 08:00:00
2,13944,2005-03-01 00:30:00,0.0,BAD,0.0,166.0,0.0,32.30,-93.40,0 days 06:00:00
3,14704,2005-03-01 00:34:00,0.0,FMH,0.0,132.0,0.0,41.40,-70.31,0 days 05:00:00
4,27401,2005-03-01 00:35:00,0.0,BTI,0.0,5.0,0.0,70.08,-143.35,0 days 09:00:00


In [45]:
# Convert Time to Utc in precipitation data
# precipitation_2005.Time = precipitation_2005.Time + Allstationsdata.TimeZone
precipitation_2005.rename(columns = {"Time":"UTC_Time"},inplace = True)

In [46]:
precipitation_2005=precipitation_2005.drop(['AirportID','GroundHeight','StationHeight','BarometerHeight','Latitude','Longitude','TimeZone'],axis = 1)

In [47]:
precipitation_2005.head()

,WeatherStationID,UTC_Time,HourlyPrecip
0,27401,2005-03-01 00:15:00,0.0
1,24201,2005-03-01 00:17:00,0.0
2,13944,2005-03-01 00:30:00,0.0
3,14704,2005-03-01 00:34:00,0.0
4,27401,2005-03-01 00:35:00,0.0


In [48]:
# Merge presipitation data with test data using origin
precipitation_2005.rename(columns = {'WeatherStationID':'Origin_ws_id',"UTC_Time":"UTC_Deptime"}, inplace = True)
precipitation_2005.head()

,Origin_ws_id,UTC_Deptime,HourlyPrecip
0,27401,2005-03-01 00:15:00,0.0
1,24201,2005-03-01 00:17:00,0.0
2,13944,2005-03-01 00:30:00,0.0
3,14704,2005-03-01 00:34:00,0.0
4,27401,2005-03-01 00:35:00,0.0


In [49]:
test["Dep_avg"] = np.array(test.UTC_Deptime, dtype='datetime64[h]')
precipitation_2005["Dep_avg"] = np.array(precipitation_2005.UTC_Deptime, dtype='datetime64[h]')
precipitation_2005=precipitation_2005.drop(['UTC_Deptime'],axis = 1)

In [50]:
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00


In [51]:
precipitation_2005.head()

,Origin_ws_id,HourlyPrecip,Dep_avg
0,27401,0.0,2005-03-01
1,24201,0.0,2005-03-01
2,13944,0.0,2005-03-01
3,14704,0.0,2005-03-01
4,27401,0.0,2005-03-01


In [52]:
test = test.merge(precipitation_2005, how='left', on=['Origin_ws_id',"Dep_avg"])

In [53]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6566 entries, 0 to 6565
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   FlightNumber         6566 non-null   object        
 1   UTC_Deptime          6566 non-null   datetime64[ns]
 2   DayOfWeek            6566 non-null   int64         
 3   UTC_Arrtime          6566 non-null   object        
 4   ScheduledTravelTime  6566 non-null   int64         
 5   Origin               6566 non-null   object        
 6   Destination          6566 non-null   object        
 7   Distance             6566 non-null   int64         
 8   Origin_ws_id         6566 non-null   int64         
 9   Destination_ws_id    6566 non-null   int64         
 10  Origin_GroH          6566 non-null   int64         
 11  Origin_StaH          6566 non-null   int64         
 12  Origin_BaroH         6566 non-null   int64         
 13  Des_GroH             6566 non-nul

In [54]:
# Rename hourly precip by considering origin in test
test.rename(columns = {'HourlyPrecip':'Origin_Hurpre'}, inplace = True) 
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0


In [55]:
# Merge presipitation data with test data using Desitination
test["Arr_avg"] = np.array(test.UTC_Arrtime, dtype='datetime64[h]')
precipitation_2005["Arr_avg"]=np.array(precipitation_2005.Dep_avg, dtype='datetime64[h]')
precipitation_2005=precipitation_2005.drop(['Dep_avg'],axis = 1)

In [56]:
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre,Arr_avg
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0,2005-09-12 23:00:00
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0,2005-09-06 17:00:00
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0,2005-07-24 00:00:00
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0,2005-09-04 21:00:00
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0,2005-03-24 00:00:00


In [57]:
precipitation_2005.head()

,Origin_ws_id,HourlyPrecip,Arr_avg
0,27401,0.0,2005-03-01
1,24201,0.0,2005-03-01
2,13944,0.0,2005-03-01
3,14704,0.0,2005-03-01
4,27401,0.0,2005-03-01


In [58]:
precipitation_2005.rename(columns = {'Origin_ws_id':'Destination_ws_id'}, inplace = True)

In [59]:
test = test.merge(precipitation_2005, how='left', on=['Destination_ws_id',"Arr_avg"])

In [60]:
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre,Arr_avg,HourlyPrecip
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0,2005-09-12 23:00:00,0.0
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0,2005-09-06 17:00:00,0.0
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0,2005-07-24 00:00:00,0.0
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0,2005-09-04 21:00:00,0.0
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0,2005-03-24 00:00:00,0.0


In [61]:
test.rename(columns = {'HourlyPrecip':'Des_Hurpre'}, inplace = True)
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre,Arr_avg,Des_Hurpre
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0,2005-09-12 23:00:00,0.0
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0,2005-09-06 17:00:00,0.0
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0,2005-07-24 00:00:00,0.0
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0,2005-09-04 21:00:00,0.0
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0,2005-03-24 00:00:00,0.0


In [62]:
#Reading 2005 weather hourly data
w7=pd.read_csv("200503hourly.txt")
w8=pd.read_csv("200507hourly.txt")
w9=pd.read_csv("200509hourly.txt")
w10=pd.read_csv("200511hourly.txt")

In [63]:
w_2005=[w7,w8,w9,w10]
hourly_2005=pd.concat(w_2005)
hourly_2005=hourly_2005.reset_index(drop=True)

In [64]:
hourly_2005.describe().apply(lambda s: s.apply('{0:0.2f}'.format))

,WeatherStationID,YearMonthDay,Time,DBT,DewPointTemp,RelativeHumidityPercent,WindGustValue,StationPressure
count,5031729.00,5031729.00,5031729.00,3931927.00,3919622.00,3916051.00,4989777.00,4401844.00
mean,37168.04,20050768.52,1185.92,57.91,44.80,66.64,2.78,29.11
std,33478.21,295.05,686.11,20.55,19.52,21.83,7.67,2.09
min,3011.00,20050301.00,10.00,-46.00,-81.00,0.00,0.00,0.00
25%,12935.00,20050701.00,556.00,43.00,30.00,51.00,0.00,29.06
50%,23275.00,20050731.00,1155.00,60.00,46.00,70.00,0.00,29.76
75%,54807.00,20051101.00,1755.00,74.00,61.00,85.00,0.00,30.01
max,96402.00,20051130.00,2359.00,127.00,90.00,100.00,298.00,31.50


In [65]:
hourly_2005.Time=hourly_2005.Time.astype(str).str.zfill(4)
hourly_2005.Time=pd.to_datetime(hourly_2005.Time, format="%H%M").dt.time
hourly_2005.head()

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure
0,4843,20050301,00:10:00,FEW008 SCT012 OVC026,1.75SM,NaN,NaN,NaN,6.0,VRB,0.0,NaN
1,13874,20050301,00:10:00,SCT023 BKN030,10SM,NaN,NaN,NaN,14.0,310,0.0,NaN
2,14765,20050301,00:10:00,FEW007 SCT010 OVC015,2SM,NaN,NaN,NaN,19.0,030,26.0,NaN
3,14777,20050301,00:10:00,OVC011,2.5SM,NaN,NaN,NaN,5.0,040,0.0,NaN
4,14819,20050301,00:10:00,FEW015 OVC060,2SM,NaN,NaN,NaN,9.0,310,0.0,NaN


In [66]:
hourly_2005.YearMonthDay = pd.to_datetime(hourly_2005.YearMonthDay, format="%Y%m%d")
hourly_2005.head()

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure
0,4843,2005-03-01,00:10:00,FEW008 SCT012 OVC026,1.75SM,NaN,NaN,NaN,6.0,VRB,0.0,NaN
1,13874,2005-03-01,00:10:00,SCT023 BKN030,10SM,NaN,NaN,NaN,14.0,310,0.0,NaN
2,14765,2005-03-01,00:10:00,FEW007 SCT010 OVC015,2SM,NaN,NaN,NaN,19.0,030,26.0,NaN
3,14777,2005-03-01,00:10:00,OVC011,2.5SM,NaN,NaN,NaN,5.0,040,0.0,NaN
4,14819,2005-03-01,00:10:00,FEW015 OVC060,2SM,NaN,NaN,NaN,9.0,310,0.0,NaN


In [67]:
hourly_2005.Time = hourly_2005.apply(lambda r : pd.datetime.combine(r['YearMonthDay'],r['Time']),1)
hourly_2005 = hourly_2005.drop(['YearMonthDay'],axis = 1)

In [68]:
# Merging hourly data of 2005 with test data
hourly_2005['visibility'] = hourly_2005.Visibility.str.extract('([-]?[\d.]*)', expand=False).str.strip()

In [69]:
hourly_2005=hourly_2005.drop(['Visibility'],axis = 1)
hourly_2005.head()

,WeatherStationID,Time,SkyConditions,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,visibility
0,4843,2005-03-01 00:10:00,FEW008 SCT012 OVC026,NaN,NaN,NaN,6.0,VRB,0.0,NaN,1.75
1,13874,2005-03-01 00:10:00,SCT023 BKN030,NaN,NaN,NaN,14.0,310,0.0,NaN,10
2,14765,2005-03-01 00:10:00,FEW007 SCT010 OVC015,NaN,NaN,NaN,19.0,030,26.0,NaN,2
3,14777,2005-03-01 00:10:00,OVC011,NaN,NaN,NaN,5.0,040,0.0,NaN,2.5
4,14819,2005-03-01 00:10:00,FEW015 OVC060,NaN,NaN,NaN,9.0,310,0.0,NaN,2


In [70]:
# Merge hourly data and test data using origin
hourly_2005.rename(columns = {"WeatherStationID":"Origin_ws_id","Time":"Dep_avg"},inplace = True)

In [71]:
hourly_2005["Dep_avg"] = np.array(hourly_2005.Dep_avg, dtype='datetime64[h]')
hourly_2005.head()

,Origin_ws_id,Dep_avg,SkyConditions,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,visibility
0,4843,2005-03-01,FEW008 SCT012 OVC026,NaN,NaN,NaN,6.0,VRB,0.0,NaN,1.75
1,13874,2005-03-01,SCT023 BKN030,NaN,NaN,NaN,14.0,310,0.0,NaN,10
2,14765,2005-03-01,FEW007 SCT010 OVC015,NaN,NaN,NaN,19.0,030,26.0,NaN,2
3,14777,2005-03-01,OVC011,NaN,NaN,NaN,5.0,040,0.0,NaN,2.5
4,14819,2005-03-01,FEW015 OVC060,NaN,NaN,NaN,9.0,310,0.0,NaN,2


In [72]:
test = test.merge(hourly_2005, how='left', on=['Origin_ws_id',"Dep_avg"])
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre,Arr_avg,Des_Hurpre,SkyConditions,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,visibility
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0,2005-09-12 23:00:00,0.0,FEW100 SCT140,55.0,34.0,45.0,9.0,120,0.0,24.63,10
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0,2005-09-06 17:00:00,0.0,FEW070 SCT120 SCT150,77.0,48.0,36.0,4.0,VRB,0.0,24.84,10
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0,2005-07-24 00:00:00,0.0,FEW070 SCT110 BKN150,71.0,54.0,55.0,10.0,300,0.0,24.73,10
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0,2005-09-04 21:00:00,0.0,FEW080 BKN140,75.0,43.0,32.0,8.0,360,0.0,24.71,10
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0,2005-03-24 00:00:00,0.0,BKN009 OVC100,NaN,NaN,NaN,4.0,190,0.0,NaN,6


In [73]:
test.rename(columns = {"DBT":"Ori_DBT","DewPointTemp":"Ori_DewPT","RelativeHumidityPercent":"Ori_RelHumiP","WindSpeed":"Ori_WindSpeed",
                        "WindDirection":"Ori_WindDir","WindGustValue":"Ori_WindGu",
                        "StationPressure":"Ori_StaPre","visibility":"Ori_Visib","SkyConditions":"Ori_SkyCon"},inplace = True)

test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre,Arr_avg,Des_Hurpre,Ori_SkyCon,Ori_DBT,Ori_DewPT,Ori_RelHumiP,Ori_WindSpeed,Ori_WindDir,Ori_WindGu,Ori_StaPre,Ori_Visib
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0,2005-09-12 23:00:00,0.0,FEW100 SCT140,55.0,34.0,45.0,9.0,120,0.0,24.63,10
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0,2005-09-06 17:00:00,0.0,FEW070 SCT120 SCT150,77.0,48.0,36.0,4.0,VRB,0.0,24.84,10
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0,2005-07-24 00:00:00,0.0,FEW070 SCT110 BKN150,71.0,54.0,55.0,10.0,300,0.0,24.73,10
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0,2005-09-04 21:00:00,0.0,FEW080 BKN140,75.0,43.0,32.0,8.0,360,0.0,24.71,10
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0,2005-03-24 00:00:00,0.0,BKN009 OVC100,NaN,NaN,NaN,4.0,190,0.0,NaN,6


In [74]:
# Merge hourly data and test using Destination
hourly_2005.rename(columns = {"Origin_ws_id":"Destination_ws_id","Dep_avg":"Arr_avg"},inplace = True)
hourly_2005.head()

,Destination_ws_id,Arr_avg,SkyConditions,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,visibility
0,4843,2005-03-01,FEW008 SCT012 OVC026,NaN,NaN,NaN,6.0,VRB,0.0,NaN,1.75
1,13874,2005-03-01,SCT023 BKN030,NaN,NaN,NaN,14.0,310,0.0,NaN,10
2,14765,2005-03-01,FEW007 SCT010 OVC015,NaN,NaN,NaN,19.0,030,26.0,NaN,2
3,14777,2005-03-01,OVC011,NaN,NaN,NaN,5.0,040,0.0,NaN,2.5
4,14819,2005-03-01,FEW015 OVC060,NaN,NaN,NaN,9.0,310,0.0,NaN,2


In [75]:
test = test.merge(hourly_2005, how='left', on=['Destination_ws_id',"Arr_avg"])
test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre,Arr_avg,Des_Hurpre,Ori_SkyCon,Ori_DBT,Ori_DewPT,Ori_RelHumiP,Ori_WindSpeed,Ori_WindDir,Ori_WindGu,Ori_StaPre,Ori_Visib,SkyConditions,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,visibility
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0,2005-09-12 23:00:00,0.0,FEW100 SCT140,55.0,34.0,45.0,9.0,120,0.0,24.63,10,SCT110,54.0,32.0,43.0,5.0,120,0.0,24.64,10
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0,2005-09-06 17:00:00,0.0,FEW070 SCT120 SCT150,77.0,48.0,36.0,4.0,VRB,0.0,24.84,10,FEW070 SCT120 BKN150,73.0,48.0,41.0,10.0,140,0.0,24.86,10
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0,2005-07-24 00:00:00,0.0,FEW070 SCT110 BKN150,71.0,54.0,55.0,10.0,300,0.0,24.73,10,SCT130 BKN180,67.0,54.0,63.0,3.0,150,0.0,24.73,10
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0,2005-09-04 21:00:00,0.0,FEW080 BKN140,75.0,43.0,32.0,8.0,360,0.0,24.71,10,FEW080 BKN120,75.0,43.0,32.0,7.0,280,0.0,24.73,10
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0,2005-03-24 00:00:00,0.0,BKN009 OVC100,NaN,NaN,NaN,4.0,190,0.0,NaN,6,BKN085,33.0,31.0,92.0,0.0,000,0.0,24.31,4


In [76]:
test.rename(columns = {"DBT":"Des_DBT","DewPointTemp":"Des_DewPT","RelativeHumidityPercent":"Des_RelHumP","WindSpeed":"Des_WindS",
                        "WindDirection":"Des_WindDir","WindGustValue":"Des_WindGust",
                        "StationPressure":"Des_Stapre","visibility":"Des_Visib","SkyConditions":"Des_SkyCon"},inplace = True)

test.head()

,FlightNumber,UTC_Deptime,DayOfWeek,UTC_Arrtime,ScheduledTravelTime,Origin,Destination,Distance,Origin_ws_id,Destination_ws_id,Origin_GroH,Origin_StaH,Origin_BaroH,Des_GroH,Des_StaH,Des_BaroH,Dep_avg,Origin_Hurpre,Arr_avg,Des_Hurpre,Ori_SkyCon,Ori_DBT,Ori_DewPT,Ori_RelHumiP,Ori_WindSpeed,Ori_WindDir,Ori_WindGu,Ori_StaPre,Ori_Visib,Des_SkyCon,Des_DBT,Des_DewPT,Des_RelHumP,Des_WindS,Des_WindDir,Des_WindGust,Des_Stapre,Des_Visib
0,DEN_XNA_62349,2005-09-12 22:16:00,1,2005-09-12 23:56:00,100,DEN,XNA,616,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-12 22:00:00,0.0,2005-09-12 23:00:00,0.0,FEW100 SCT140,55.0,34.0,45.0,9.0,120,0.0,24.63,10,SCT110,54.0,32.0,43.0,5.0,120,0.0,24.64,10
1,DEN_SAT_82515,2005-09-06 15:55:00,2,2005-09-06 17:57:00,122,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-06 15:00:00,0.0,2005-09-06 17:00:00,0.0,FEW070 SCT120 SCT150,77.0,48.0,36.0,4.0,VRB,0.0,24.84,10,FEW070 SCT120 BKN150,73.0,48.0,41.0,10.0,140,0.0,24.86,10
2,DEN_ICT_94174,2005-07-23 22:56:00,6,2005-07-24 00:17:00,81,DEN,ICT,419,3017,3017,5379,5431,5382,5379,5431,5382,2005-07-23 22:00:00,0.0,2005-07-24 00:00:00,0.0,FEW070 SCT110 BKN150,71.0,54.0,55.0,10.0,300,0.0,24.73,10,SCT130 BKN180,67.0,54.0,63.0,3.0,150,0.0,24.73,10
3,DEN_SAT_78504,2005-09-04 19:30:00,7,2005-09-04 21:29:00,119,DEN,SAT,794,3017,3017,5379,5431,5382,5379,5431,5382,2005-09-04 19:00:00,0.0,2005-09-04 21:00:00,0.0,FEW080 BKN140,75.0,43.0,32.0,8.0,360,0.0,24.71,10,FEW080 BKN120,75.0,43.0,32.0,7.0,280,0.0,24.73,10
4,DEN_PHL_82820,2005-03-23 21:05:00,3,2005-03-24 00:25:00,200,DEN,PHL,1557,3017,3017,5379,5431,5382,5379,5431,5382,2005-03-23 21:00:00,0.0,2005-03-24 00:00:00,0.0,BKN009 OVC100,NaN,NaN,NaN,4.0,190,0.0,NaN,6,BKN085,33.0,31.0,92.0,0.0,000,0.0,24.31,4


In [77]:
test=test.replace("NAN",np.nan)
test.replace("?",np.nan,inplace=True)
test.replace("???",np.nan,inplace=True)
test.replace("MISSINGVALUE",np.nan,inplace=True)
test.replace("MISSEDDATA",np.nan,inplace=True)
test.replace(-1,np.nan,inplace=True)
test.replace(-5,np.nan,inplace=True)
test.replace("MISSINGVAL",np.nan,inplace=True)
test.replace("VRB",np.nan,inplace=True)

In [78]:
test.isnull().sum()

FlightNumber              0
UTC_Deptime               0
DayOfWeek                 0
UTC_Arrtime               0
ScheduledTravelTime       0
Origin                    0
Destination               0
Distance                  0
Origin_ws_id              0
Destination_ws_id         0
Origin_GroH               0
Origin_StaH               0
Origin_BaroH              0
Des_GroH                  0
Des_StaH                  0
Des_BaroH                 0
Dep_avg                   0
Origin_Hurpre          1027
Arr_avg                   0
Des_Hurpre             1037
Ori_SkyCon               25
Ori_DBT                1871
Ori_DewPT              1875
Ori_RelHumiP           1875
Ori_WindSpeed            27
Ori_WindDir             328
Ori_WindGu               27
Ori_StaPre             1866
Ori_Visib                25
Des_SkyCon               56
Des_DBT                1735
Des_DewPT              1739
Des_RelHumP            1737
Des_WindS                54
Des_WindDir             297
Des_WindGust        

In [79]:
missing_test = test.isnull().sum(axis=0).reset_index()
missing_test.columns = ['variable', 'missing values']
missing_test['filling factor (%)']=(test.shape[0]-missing_test['missing values'])/test.shape[0]*100
missing_test.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,Ori_DewPT,1875,79.823523
1,Ori_RelHumiP,1875,79.823523
2,Ori_DBT,1871,79.866566
3,Ori_StaPre,1866,79.920370
4,Des_DewPT,1739,81.286990
5,Des_RelHumP,1737,81.308512
6,Des_DBT,1735,81.330033
7,Des_Stapre,1731,81.373077
8,Des_Hurpre,1037,88.841063
9,Origin_Hurpre,1027,88.948671


In [80]:
test['Ori_Visib']=test['Ori_Visib'].fillna(0)
test['Ori_WindDir']=test['Ori_WindDir'].fillna(0)
test['Ori_WindGu']=test['Ori_WindGu'].fillna(0)
test['Ori_WindSpeed']=test['Ori_WindSpeed'].fillna(0)
test['Des_WindS']=test['Des_WindS'].fillna(0)
test['Des_WindDir']=test['Des_WindDir'].fillna(0)
test['Des_WindGust']=test['Des_WindGust'].fillna(0)
test['Des_Visib']=test['Des_Visib'].fillna(0)
test['Ori_SkyCon']=test['Ori_SkyCon'].fillna(0)
test['Des_SkyCon']=test['Des_SkyCon'].fillna(0)

In [81]:
test['Ori_StaPre'].fillna(test['Ori_StaPre'].mean(),inplace=True)
test['Des_DBT'].fillna(test['Des_DBT'].mean(),inplace=True)
test['Des_DewPT'].fillna(test['Des_DewPT'].mean(),inplace=True)
test['Des_RelHumP'].fillna(test['Des_RelHumP'].mean(),inplace=True)
test['Des_Stapre'].fillna(test['Des_Stapre'].mean(),inplace=True)

In [83]:
test['Ori_RelHumiP']=pd.to_numeric(test['Ori_RelHumiP'])
ORHP_mean_value=test['Ori_RelHumiP'].mean()
test['Ori_RelHumiP'].fillna(value=ORHP_mean_value, inplace=True)


test['Ori_DewPT']=pd.to_numeric(test['Ori_DewPT'])
ODEPT_mean_value=test['Ori_DewPT'].mean()
test['Ori_DewPT'].fillna(value=ODEPT_mean_value, inplace=True)


test['Ori_DBT']=pd.to_numeric(test['Ori_DBT'])
ODBT_mean_value=test['Ori_RelHumiP'].mean()
test['Ori_DBT'].fillna(value=ODBT_mean_value, inplace=True)


test['Des_Hurpre']=pd.to_numeric(test['Des_Hurpre'])
DHP_mean_value=test['Des_Hurpre'].mean()
test['Des_Hurpre'].fillna(value=DHP_mean_value, inplace=True)


test['Origin_Hurpre']=pd.to_numeric(test['Origin_Hurpre'])
OHP_mean_value=test['Origin_Hurpre'].mean()
test['Origin_Hurpre'].fillna(value=OHP_mean_value, inplace=True)

In [84]:
test.isnull().sum()

FlightNumber           0
UTC_Deptime            0
DayOfWeek              0
UTC_Arrtime            0
ScheduledTravelTime    0
Origin                 0
Destination            0
Distance               0
Origin_ws_id           0
Destination_ws_id      0
Origin_GroH            0
Origin_StaH            0
Origin_BaroH           0
Des_GroH               0
Des_StaH               0
Des_BaroH              0
Dep_avg                0
Origin_Hurpre          0
Arr_avg                0
Des_Hurpre             0
Ori_SkyCon             0
Ori_DBT                0
Ori_DewPT              0
Ori_RelHumiP           0
Ori_WindSpeed          0
Ori_WindDir            0
Ori_WindGu             0
Ori_StaPre             0
Ori_Visib              0
Des_SkyCon             0
Des_DBT                0
Des_DewPT              0
Des_RelHumP            0
Des_WindS              0
Des_WindDir            0
Des_WindGust           0
Des_Stapre             0
Des_Visib              0
dtype: int64

In [85]:
test['y']=test['UTC_Deptime']
test['y']=test['y'].dt.round('60min')

In [86]:
test=test.drop_duplicates(subset=['FlightNumber','y'],keep='last').reset_index(drop=True)

In [87]:
test = test.to_csv('testresult.csv')